In [63]:
from bs4 import BeautifulSoup

In [89]:
def find_nth(string, substring, n):
    if (n == 1):
        return string.find(substring)
    else:
        return string.find(substring, find_nth(string, substring, n - 1) + 1)
    
items = ["Item 1.", "Item 1A.", "Item 1B.", "Item 2.", "Item 3.", 
         "Item 4.","Item 5.", "Item 6.","Item 7.","Item 7A.","Item 8.",
         "Item 9.","Item 9A.","Item 9B.","Item 10.","Item 11.","Item 12.",
         "Item 13.","Item 14.", "Item 15.","Item 16.","Signatures" ]

def only_occurs_twice(text, item):
    # shouldn't have to check for upper case because upper case ITEM won't occur in-text 
    if (find_nth(text, item, 3) != -1):
        return False
    else:
        return True 

def parse_10k(text,items):
    sections = {}
    for i in range(len(items)-1):
        section_start_index = find_nth(text,items[i],2)
        if (section_start_index == -1):
            # Also try upper case 
            print("trying upper case..{}".format(items[i].upper()))
            section_start_index = find_nth(text, items[i].upper(), 1)

        section_end_index = find_nth(text,items[i+1],2) 
        if (section_end_index == -1):
            # Also try upper case 
            section_end_index = find_nth(text, items[i+1].upper(), 1)
        
        if ( (section_start_index != -1) and (section_end_index != -1)  ):
            sections[items[i]] = text[section_start_index:section_end_index]
        else:
            print("Something went wrong for {}".format(items[i]))
            print(section_start_index,section_end_index)
            sections[items[i]] = ""
            
        # Process for 10-Ks without Item 16 
        if ( (items[i+1] == "Item 16.") and (section_end_index == -1 ) ):
            # Just skip to Signatures
            section_end_index = find_nth(text,items[i+2],2)
            
            # try upper case
            if (section_end_index == -1):
                section_end_index = find_nth(text,items[i+2].upper(),1)
           
            if ( (section_start_index != -1) and (section_end_index != -1)  ):
                sections[items[i]] = text[section_start_index:section_end_index]
            else:
                print("Something went wrong for {}".format(items[i]))
                print(section_start_index,section_end_index)
                sections[items[i]] = ""
                break 
        
    return sections 
    
    
def process_10k(html_file):
    soup_raw = BeautifulSoup(open(html_file), 'html.parser')
    text = soup_raw.get_text()
    text = text.replace("\n"," ")
    text = text.replace("\xa0"," ") 
    
    sections = parse_10k(text, items) 
    return sections

def print_sections(sections):
    for item in items:
        print("key is {}".format(item))
        if (item in sections.keys()):
            if (item != "SignatureS"):
                print(sections[item][:150])
                print("--")
                print(sections[item][-150:])
                print("------------------------")
        

In [85]:
msft = process_10k("MSFT 10K 2018 630.htm")
msft["Item 1."][:100]

trying upper case..ITEM 1.
trying upper case..ITEM 1A.
trying upper case..ITEM 1B.
trying upper case..ITEM 2.
trying upper case..ITEM 3.
trying upper case..ITEM 4.
trying upper case..ITEM 5.
trying upper case..ITEM 6.
trying upper case..ITEM 7.
trying upper case..ITEM 7A.
trying upper case..ITEM 8.
trying upper case..ITEM 9.
trying upper case..ITEM 9B.
trying upper case..ITEM 10.
trying upper case..ITEM 11.
trying upper case..ITEM 12.
trying upper case..ITEM 13.
trying upper case..ITEM 14.
trying upper case..ITEM 15.
trying upper case..ITEM 16.


'ITEM 1. BUSINESS  GENERAL  Embracing Our Future Microsoft is a technology company whose mission is t'

In [86]:
amzn = process_10k("AMZN 10K 2007.html")

Something went wrong for Item 15.
269754 -1
trying upper case..ITEM 16.
Something went wrong for Item 16.
-1 274689


In [90]:
print_sections(amzn)

key is Item 1.
Item 1.    Business   This Annual Report on Form 10-K and the documents incorporated herein by reference contain forward-looking statements based on e
--
red, Former Chairman, Reader’s Digest Association, Inc.   Patricia Q. Stonesifer    51     Chief Executive Officer, Bill & Melinda Gates Foundation   
------------------------
key is Item 1A.
Item 1A.    Risk Factors  Please carefully consider the following risk factors. If any of the following risks occur, our business, financial condition
--
n unlawful manner, or violating the proprietary rights of others, and could face civil or criminal liability for unlawful activities by our sellers.  
------------------------
key is Item 1B.
Item 1B.     Unresolved Staff Comments  None.  
--
Item 1B.     Unresolved Staff Comments  None.  
------------------------
key is Item 2.
Item 2.     Properties  We do not own any real estate. As of December 31, 2007, we operated the following facilities:               Description of Use
--


In [91]:
nflx = process_10k("NFLX 10K 2006.html")
print_sections(nflx)

Something went wrong for Item 15.
168109 -1
trying upper case..ITEM 16.
Something went wrong for Item 16.
-1 250099
key is Item 1.
Item 1. Business  We are the largest online movie rental subscription service providing more than 6,300,000 subscribers access to a comprehensive libr
--
easonably practicable after electronically filing or furnishing those reports to the Securities and Exchange Commission.    8    Table of Contents    
------------------------
key is Item 1A.
Item 1A. Risk Factors  If any of the following risks actually occurs, our business, financial condition and results of operations could be harmed. In 
--
nd operating data may differ materially from actual results. Such discrepancies could cause a decline in the trading price of our common stock.       
------------------------
key is Item 1B.
Item 1B. Unresolved Staff Comments  None.       
--
Item 1B. Unresolved Staff Comments  None.       
------------------------
key is Item 2.
Item 2. Properties  We do not own an